In [12]:
!pip install opencv-python-headless --quiet

In [14]:
import json
from pathlib import Path
from argparse import ArgumentParser
import requests
import os, cv2
import zipfile
import numpy as np
import shutil

In [3]:
annotations_path = "./annotation_val.odgt"

In [4]:
target_class= ["vbody"]
crowdHuman_path = "val/"
datasetPath = "yolo_crowd_dataset/"
imgPath = "images/"
labelPath = "labels/"
imgType = ".jpg"

In [5]:
def generateYOLO(filename, bbox):
    file = open(os.path.join(datasetPath, labelPath, filename+'.txt'), "w")
    file.writelines(bbox)
    file.close

In [6]:
def convert(size, box, img_id):
    if box[0] > size[0] or box[2] > size[0] or box[1] > size[1] or box[3] > size[1]:  
        ###wrong labelling, label coordinate is bigger than or overside the image size
        w = open(os.path.join(datasetPath, 'tagging_box_error'), 'a')
        w.write(img_id+'\n')
        w.close
        return [''] ###return nothing, give up this label
    dw = 1./(size[0])
    dh = 1./(size[1])
    xmax = abs(box[0]) + abs(box[2])
    ymax = abs(box[1]) + abs(box[3])
    if xmax > size[0]:  ###avoid the label box out of the image
        xmax = size[0]
    if ymax > size[1]:
        ymax = size[1]
    x = (abs(box[0]) + xmax)/2.0 - 1
    y = (abs(box[1]) + ymax)/2.0 - 1
    w = abs(box[2])
    h = abs(box[3])
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return [str(0), ' ', str(round(x, 6)),' ', str(round(y, 6)), ' ', str(round(w, 6)), ' ', str(round(h, 6)), '\n'] 
    ###the first one, index 0, is the class id and assuem "person" class is 0


In [ ]:
f = open(annotations_path)
lines = f.readlines()
total_lines = len(lines)

for lineID, line in enumerate(lines):
    data = eval(line)
    img_id = data["ID"]
    
    image_path = os.path.join(crowdHuman_path, img_id+'.jpg')
    if os.path.exists(image_path):
        img = cv2.imread(image_path)
        #cv2.imwrite(os.path.join(datasetPath, imgPath, img_id + imgType), img) ###uncomment to generate image
        (img_height, img_width, img_depth) = img.shape
        total_iid = len(data["gtboxes"])       
        box_list=[]

        for iid, infoBody in enumerate(data["gtboxes"]):
            if infoBody['tag'] != 'mask':
                tag = infoBody["tag"]
                hbox = infoBody["hbox"]
                head_attr = infoBody["head_attr"]
                fbox = infoBody["fbox"]
                vbox = infoBody["vbox"]
                extra = infoBody["extra"]
                difficult = 0

                if(len(target_class)==0 or ("head" in target_class)):
                    box_list.extend(convert((img_width, img_height), hbox, img_id))

                if(len(target_class)==0 or ("fbody" in target_class)):
                    box_list.extend(convert((img_width, img_height), fbox, img_id))

                if(len(target_class)==0 or ("vbody" in target_class)):
                    box_list.extend(convert((img_width, img_height), vbox, img_id))
        
                print("[Left over] {}/{} , {}: head:{} full:{} visible:{}".format(total_lines-lineID, total_iid-iid, img_id,\
                    hbox, fbox, vbox)) #end of one label object loop 
        generateYOLO(img_id, box_list) #end of one image loop
    else:
        f = open(os.path.join(datasetPath, 'no_this_image_log'), 'a')  ###can't find the label file name in imgPath
        f.write(img_id+'\n')
        f.close


In [ ]:
!aws s3 cp s3://chdataset/crowdhuman_dataset/images/val/ ./val --recursive

In [ ]:
!aws s3 cp /home/sagemaker-user/yolo_crowd_dataset/labels/ s3://chdataset/crowdhuman_dataset/labels/val/

In [5]:
import boto3, os, sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [ ]:
path = "yolo_crowd_dataset/labels/"
s3 = boto3.client('s3')
sess = sagemaker.Session()
smclient = boto3.Session().client('sagemaker')

In [24]:
path = "yolo_crowd_dataset/labels/"
s3 = boto3.client('s3')
sess = sagemaker.Session()

if os.path.exists(path):
    file_lists = os.listdir(path)

    for file in file_lists:
        if file.endswith(".txt"):
            sess.upload_data(path+file, bucket = 'chdataset',key_prefix = "crowdhuman_dataset/labels/val")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
!aws s3 cp s3://chdataset/crowdhuman_dataset/images/Images.zip .

download: s3://chdataset/crowdhuman_dataset/images/Images.zip to ./Images.zip


In [ ]:
!unzip Images.zip

In [6]:
path = "Images/"
s3 = boto3.client('s3')
sess = sagemaker.Session()

if os.path.exists(path):
    file_lists = os.listdir(path)
    for file in file_lists:
        if file.endswith(".jpg"):
            sess.upload_data(path+file, bucket = 'chdataset',key_prefix = "crowdhuman_dataset/images/train")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [16]:
annotations_path = "./annotation_train.odgt"

In [17]:
target_class= ["vbody"]
crowdHuman_path = "Images/"
datasetPath = "yolo_crowd_dataset/"
imgPath = "images/"
labelPath = "labels/"
trainlabelPath = "train_labels/"
imgType = ".jpg"

In [18]:
def generateYOLO(filename, bbox):
    file = open(os.path.join(datasetPath, trainlabelPath, filename+'.txt'), "w")
    file.writelines(bbox)
    file.close

In [19]:
def convert(size, box, img_id):
    if box[0] > size[0] or box[2] > size[0] or box[1] > size[1] or box[3] > size[1]:  
        ###wrong labelling, label coordinate is bigger than or overside the image size
        w = open(os.path.join(datasetPath, 'tagging_box_error'), 'a')
        w.write(img_id+'\n')
        w.close
        return [''] ###return nothing, give up this label
    dw = 1./(size[0])
    dh = 1./(size[1])
    xmax = abs(box[0]) + abs(box[2])
    ymax = abs(box[1]) + abs(box[3])
    if xmax > size[0]:  ###avoid the label box out of the image
        xmax = size[0]
    if ymax > size[1]:
        ymax = size[1]
    x = (abs(box[0]) + xmax)/2.0 - 1
    y = (abs(box[1]) + ymax)/2.0 - 1
    w = abs(box[2])
    h = abs(box[3])
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return [str(0), ' ', str(round(x, 6)),' ', str(round(y, 6)), ' ', str(round(w, 6)), ' ', str(round(h, 6)), '\n'] 
    ###the first one, index 0, is the class id and assuem "person" class is 0


In [ ]:
f = open(annotations_path)
lines = f.readlines()
total_lines = len(lines)

for lineID, line in enumerate(lines):
    data = eval(line)
    img_id = data["ID"]
    
    image_path = os.path.join(crowdHuman_path, img_id+'.jpg')
    if os.path.exists(image_path):
        img = cv2.imread(image_path)
        #cv2.imwrite(os.path.join(datasetPath, imgPath, img_id + imgType), img) ###uncomment to generate image
        (img_height, img_width, img_depth) = img.shape
        total_iid = len(data["gtboxes"])       
        box_list=[]

        for iid, infoBody in enumerate(data["gtboxes"]):
            if infoBody['tag'] != 'mask':
                tag = infoBody["tag"]
                hbox = infoBody["hbox"]
                head_attr = infoBody["head_attr"]
                fbox = infoBody["fbox"]
                vbox = infoBody["vbox"]
                extra = infoBody["extra"]
                difficult = 0

                if(len(target_class)==0 or ("head" in target_class)):
                    box_list.extend(convert((img_width, img_height), hbox, img_id))

                if(len(target_class)==0 or ("fbody" in target_class)):
                    box_list.extend(convert((img_width, img_height), fbox, img_id))

                if(len(target_class)==0 or ("vbody" in target_class)):
                    box_list.extend(convert((img_width, img_height), vbox, img_id))
        
                print("[Left over] {}/{} , {}: head:{} full:{} visible:{}".format(total_lines-lineID, total_iid-iid, img_id,\
                    hbox, fbox, vbox)) #end of one label object loop 
        generateYOLO(img_id, box_list) #end of one image loop
    else:
        f = open(os.path.join(datasetPath, 'no_this_image_log'), 'a')  ###can't find the label file name in imgPath
        f.write(img_id+'\n')
        f.close


In [23]:
image_path = os.path.join(datasetPath, trainlabelPath)
print(image_path)
num_files = len([f for f in os.listdir(image_path)])
print(num_files)

yolo_crowd_dataset/train_labels/
15000


In [24]:
path = "yolo_crowd_dataset/train_labels/"
s3 = boto3.client('s3')
sess = sagemaker.Session()

if os.path.exists(path):
    file_lists = os.listdir(path)

    for file in file_lists:
        if file.endswith(".txt"):
            sess.upload_data(path+file, bucket = 'chdataset',key_prefix = "crowdhuman_dataset/labels/train")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [25]:
print("DONE!")

DONE!
